In [1]:
# Imports
from typing import Iterable, TypedDict
import pandas as pd
import requests
from datetime import datetime
import pytse_client as tse
import fetch
import config
from utils import debug_log, data_log, ar_to_fa_series, ar_to_fa, fa_to_ar, fa_to_ar_series#, flatten_json

pd.set_option('display.max_columns', 50) # pd.options.display.max_columns = 3
pd.set_option('display.max_rows', 7) # pd.options.display.max_rows = 3
pd.set_option('display.float_format', '{:.2f}'.format) # = pd.options.display.float_format = '{:.0f}'.format
pd.options.display.max_colwidth = 30

#### Common Parameters

In [2]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36",
    "Referer": "http://main.tsetmc.com/StaticContent/WebServiceHelp",
}
cookie_jar = {"ASP.NET_SessionId": "wa40en1alwxzjnqehjntrv5j"}
test_ids = ['46348559193224090','20024911381434086','22129017544200','94378630649293']
# currentDateTime = datetime.now().strftime("%Y-%m-%d %H-%M-%S %p")

last_date = fetch._last_date
catalogue = fetch.get_catalogue()
# identities_df = fetch.get_identities()

insCodes = catalogue.index
insCodes_indices_excluded = catalogue[catalogue['cComVal'] != '6'].index

print(f'Last Date: {last_date}')
print('Instruments: ', len(insCodes))
print('Instruments (Excluding Indices): ', len(insCodes_indices_excluded))
print('Indices: ', len(insCodes) - len(insCodes_indices_excluded))

Last Date: 20230919
Instruments:  3211
Instruments (Excluding Indices):  3097
Indices:  114


In [3]:
catalogue

,instrumentID,cValMne,lVal18,cSocCSAC,lVal18AFC,lVal30,cIsin,dEven,flow,lSoc30,CGdSVal,cgrValCot,yMarNSC,cComVal,cSecVal,cSoSecVal,yVal,lSecVal,lSoSecVal,cgrValCotTitle,daily
insCode,,,,,,,,,,,,,,,,,,,,,
22129017544200,IRR1AZIN0101,AZIX1,Saipa Azin-R,AZIN,خاذینح,ح . سایپاآذین,IRR1AZIN0106,20200516,1,سایپاآذین,A,N2,NO,5,34,3499,400,خودرو و ساخت قطعات,ساير قطعات يدكي و وسايل نق...,بازار دوم بورس,20230919
94378630649293,IRB4SO110441,O11Q1,SOV.GMR110SP250622,SO11,اراد110,مرابحه عام دولت110-ش.خ040401,IRB4SO110443,20230607,2,مرابحه عام دولت110-ش.خاص0404,A,17,NO,4,69,6945,706,اوراق تامين مالي,اوراق مرابحه دولتي با ضمان...,-',20230919
114312662654155,IRO3SAIZ0001,SAIZ1,SepehrSaderat Inv.,SAIZ,وسپهر,سرمایه گذاری مالی سپهرصادرات,IRO3SAIZ0007,20230917,2,سرمایه گذاری گروه سپهرصادرات,A,1Z,NO,1,56,5699,303,سرمايه گذاريها,ساير واسطه هاي مالي,بازار اول فرابورس,20230919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72020135886048576,IRR1KABZ0101,KABX1,Alborz Carton-R,KABZ,چکارلح,ح . کارتن البرز,IRR1KABZ0106,20081126,1,کارتن البرز,A,N4,NO,4,21,2102,400,محصولات كاغذي,توليد كاغذ و مقواي موجدار ...,بازار چهارم بورس,20230919
72044846109864381,IRO3ARTZ0001,ARTZ1,Artavil Tire,ARTZ,پارتا,مجتمع صنعتی آرتاویل تایر,IRO3ARTZ0005,20230917,2,مجتمع صنعتی آرتاویل تایر,A,Z1,NO,3,25,2511,303,لاستيك و پلاستيك,توليد تاير و بازسازي تايره...,بازار دوم فرابورس,20230919
72052393871894367,IRS3ZOBZ93A1,ZO3A1,Zob-9310,ZOBZ,طذوب310,اختیارفروش تبعی ذوب آهن اص...,IRS3ZOBZ93A4,20140701,2,سهامی ذوب آهن اصفهان,A,U1,NO,8,27,2710,601,فلزات اساسي,توليد آهن و فولاد پايه,بازار ابزارهای مشتقه,20230919


In [6]:
## Getting Prices
daily_quotes = fetch.get_closing_prices_daily('all', traded_instruments_only=True)
print(f'{len(daily_quotes.keys())} Tickers traded in the last workday.')

890 Tickers traded in the last workday.


In [7]:
catalogue.to_csv(f'{config.db_path}/catalogue.csv', quoting=1)

## Categorization

In [4]:
def filter_tickers_by_industry(cSecVal: int) -> list:
    return list(catalogue[catalogue['cSecVal'] == cSecVal].index)

In [5]:
khodroi = filter_tickers_by_industry(34)

## Volume Filter

In [7]:
def volume_filter(insCodes: list) -> list:
    
    daily_prices = fetch.get_closing_prices_daily(insCodes)
    
    list_of_dfs = []
    ticker_names= []
    for insCode in insCodes:
        last_row = daily_prices[insCode].iloc[-1]
        if (
            last_row['<VOL>'] > last_row['<VOL> MA 3'] and 
            last_row['<VOL>'] > last_row['<VOL> MA 5'] and
            last_row['<VOL>'] > last_row['<VOL> MA 9'] and
            last_row['<VOL>'] > last_row['<VOL> MA 26'] and
            last_row['<VOL>'] > last_row['<VOL> MA 52'] and
            last_row['<VOL>'] > last_row['<VOL> MA 90']
        ):
            list_of_dfs.append(daily_prices[insCode])
            ticker_names.append(insCode)
            debug_log.info(f'{insCode} Filtered For Volume.')
    print(len(ticker_names),'\n')
    return ticker_names


In [8]:
volume_filter(list(daily_quotes.keys()))

114 



['490987973229371',
 '1050751214677134',
 '1438514795814416',
 '1931037287459558',
 '2944500421562364',
 '3493306453706327',
 '3577388800305243',
 '3654864906585643',
 '3846143218462419',
 '3863538898378476',
 '4384288570322406',
 '5305844922895340',
 '5599691633622269',
 '6792309561698443',
 '7745894403636165',
 '9546211590069761',
 '9925324137489171',
 '10795723506538053',
 '10831074117626896',
 '11800678942469896',
 '12965822877128721',
 '13243992182070788',
 '13281937213456378',
 '16553062355259729',
 '17933573078185644',
 '17939384202383793',
 '19298748452450329',
 '19348717261145458',
 '22086876724551482',
 '22382156782768756',
 '22424135367941584',
 '22667016906590506',
 '24085906177899789',
 '24869832924911721',
 '25336820825905643',
 '26780282166315918',
 '27299841173245405',
 '27308217070238237',
 '28291104595448527',
 '28431095903407567',
 '28725056634066391',
 '29974853866926823',
 '30231789123900526',
 '30282299500988269',
 '30507152381699953',
 '31049085025064185',
 '3152